# Часть 1. Какая модель лучше?

## Получаем свой корпус текстов

Скачиваем тексты с http://fishkamchatka.ru

In [1]:
NEWSPAPER_URL = 'http://fishkamchatka.ru/articles/exclusive/'
START_NUM = 15000
END_NUM = 30001
FILE_NAME = 'fishkamchatka.txt'

def download_page(i):
    page_link = NEWSPAPER_URL + str(i) + '/'
    print(page_link)
    r = requests.get(page_link)
    if r.status_code != 200:
        print(r.status_code)
        return
    page = html.fromstring(r.text)
    elements = page.xpath("//div[@class='detail_text_news']//text()")
    text = ""
    for element in elements:
        element = element.strip()
        if element:
            text += element.strip() + '\n'
    return text

def download_corpus():
    with open(FILE_NAME, 'w', encoding='utf-8', newline='') as fout:
        for i in range(START_NUM, END_NUM):
            text = download_page(i)
            fout.write(text.strip() + '\n')
#раскомментировать, чтобы запустить
#download_corpus()

Делаем импорты, определяем вспомогательные функции

In [164]:
from sklearn.model_selection import cross_val_score
import pandas as pd
from lxml import html
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import gensim
import numpy as np
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.metrics.pairwise import cosine_distances
import math


morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word != '' and word not in stops]

    return ' '.join(words)

Разбиваем предложения из полученного корпуса на токены, нормализуем, удаляем стоп-слова

In [3]:
def normalize_data(corpus_filename):
    data_norm = []
    with open(corpus_filename, 'r', encoding='utf-8') as fin:
        for index, line in enumerate(fin):
            data_norm.append(normalize(line))
            if index % 100 == 0:
                print(str(index) + ' analyzed')

    return data_norm
#Раскомментировать, чтобы выполнить
#data_norm = normalize_data(FILE_NAME)
#Сохраним полученный список в pickle
#with open('data_norm.pkl', 'wb') as f:
    #pickle.dump(data_norm, f)
#Прочитаем из pickle
with open('data_norm.pkl', 'rb') as f:
    data_norm = pickle.load(f)

In [4]:
data_norm[:5]

['рыбопромышленный предприятие регион продолжать наращивать объём вылов производство продукция минприрода хабаровский край рассказать январь май 2016 год компания добыть 154,1 тысяча тонна водный биоресурсы это 1,9 уровень один пять месяц прошлое год',
 'председатель комитет рыбный хозяйство сергей рябченко отметить основной усилие промысловик направить вылов нерестовый сельдь охотский мор также добыча палтус треск терпуг кальмар минтай ближний время предприятие готовиться приступить лососёвый путин',
 'переработка рыба также продолжать набирать оборот январь-май достигнуть 117 тысяча тонна готовый продукция значительно вырасти объём выпуск рыбопродукт высокий степень переработка 52,2 процент результат общий стоимость произвести продукция возрасти 14,9 процент составить 6,4 миллиард рубль отметить сергей рябченко',
 'информация пресс-служба региональный правительство успешный работа предприятие позволить 1,6 раз увеличить налоговый платёж рыбохозяйственный отрасль бюджет хабаровский кр

In [5]:
len(data_norm)

219492

## Получим word2vec-модель из нашего корпуса

In [6]:
#раскомментировать, чтобы пересчитать
#model_fish = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)

Сохраним модель в файл

In [10]:
#раскомментировать, чтобы пересчитать
#model_fish.save('model_fish.bin')

Загрузим модель из файла

In [11]:
model_fish = gensim.models.Word2Vec.load('model_fish.bin')

In [29]:
model_fish.wv.most_similar('лосось')

[('атлантический', 0.7410653233528137),
 ('тихоокеанский', 0.7378786206245422),
 ('нерка', 0.7221104502677917),
 ('кижучить', 0.7164394855499268),
 ('сёмга', 0.7122551202774048),
 ('дикий', 0.701239287853241),
 ('горбуша', 0.6971409916877747),
 ('лососёвый', 0.684512734413147),
 ('кета', 0.6739604473114014),
 ('чавыча', 0.6726918816566467)]

In [13]:
model_fish.wv.most_similar('лодка')

[('резиновый', 0.9333532452583313),
 ('моторный', 0.8869612216949463),
 ('катер', 0.8731940388679504),
 ('мотор', 0.8579673171043396),
 ('надувной', 0.8328689336776733),
 ('снасть', 0.8186315894126892),
 ('весельный', 0.8129083514213562),
 ('бредень', 0.8106197714805603),
 ('лодочный', 0.8030439019203186),
 ('квадроцикл', 0.7973795533180237)]

In [34]:
model_fish.wv.most_similar('человек')

[('ребёнок', 0.7645065784454346),
 ('семья', 0.7412943243980408),
 ('голодать', 0.7399080395698547),
 ('пожилое', 0.7390650510787964),
 ('радоваться', 0.7368481159210205),
 ('малообеспеченный', 0.7338917255401611),
 ('набраться', 0.7308886051177979),
 ('кормилец', 0.7292143106460571),
 ('соотечественник', 0.7248889207839966),
 ('барыш', 0.7243375182151794)]

## Получим модель из rusvectores

In [14]:
# возьмем ту же tayga_upos_skipgram_300_2_2019, что в примере
model_rusvectores = gensim.models.KeyedVectors.load_word2vec_format('taiga_w2v_model/model.bin', binary=True)

In [30]:
model_rusvectores.most_similar('лосось_NOUN')

[('лосось_VERB', 0.7337756156921387),
 ('горбуша_NOUN', 0.7176412343978882),
 ('семг_NOUN', 0.6928519010543823),
 ('осетр_NOUN', 0.6839677095413208),
 ('рыба_NOUN', 0.661090075969696),
 ('корюшка_NOUN', 0.6546792984008789),
 ('форель_NOUN', 0.6532124280929565),
 ('хариус_NOUN', 0.6493543982505798),
 ('тунец_NOUN', 0.6446344256401062),
 ('кета_NOUN', 0.6374989151954651)]

In [36]:
model_rusvectores.most_similar('лодка_NOUN')

[('шлюпка_NOUN', 0.8258950114250183),
 ('катер_NOUN', 0.8242191076278687),
 ('баркас_NOUN', 0.821143627166748),
 ('лодка_PROPN', 0.7729498147964478),
 ('плот_NOUN', 0.7696501612663269),
 ('лодчонка_NOUN', 0.7536875605583191),
 ('моторка_NOUN', 0.7455837726593018),
 ('суденышко_NOUN', 0.7437942028045654),
 ('катамаран_NOUN', 0.7437747716903687),
 ('ялик_NOUN', 0.7422992587089539)]

In [37]:
model_rusvectores.most_similar('человек_NOUN')

[('че-ловек_NOUN', 0.7968202829360962),
 ('чело-век_NOUN', 0.7886255979537964),
 ('лю-дь_NOUN', 0.7594989538192749),
 ('человек_PROPN', 0.7243179082870483),
 ('людей_NOUN', 0.7150776386260986),
 ('люди_NOUN', 0.6920535564422607),
 ('челове-ка_NOUN', 0.6525561213493347),
 ('человека_NOUN', 0.6498527526855469),
 ('.человек_NOUN', 0.5669378042221069),
 ('человеколо_NOUN', 0.5657871961593628)]

## Прочитаем корпус парафразов

In [18]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [19]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [49]:
data.head(2)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...


In [22]:
len(data['text_1_norm'])

7227

Будем предсказывать лейблы

In [23]:
y = data['label'].values

## Векторизуем тексты парафразов с помощью моделей

Поскольку модель rusvectores имеет частеречные тэги при словах, чтобы применять ее к тексту, нужно для каждого слова текста получить аналогичный тэг (возьмем mystem, как в задании)

In [24]:
from pymystem3 import Mystem
m = Mystem()
mapping = {}

for line in open('ru-rnc.map.txt'):
    ms, ud = line.strip('\n').split()
    mapping[ms] = ud

def normalize_mystem(text):
    tokens = []
    norm_words = m.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue

        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            if lemma in stops:
                continue
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma + '_' + pos)

    return tokens

In [25]:
normalize_mystem('Рыбу ловят с лодки.')

['рыба_NOUN', 'ловить_VERB', 'лодка_NOUN']

Функция для получения векторов для текста из заданной модели (можно передавать нашу, можно готовую).

In [26]:
def get_embedding(text, model, dim, is_mystem):
    # is_mystem указывает на то, как разбить текст на токены
    if is_mystem:
        tokens = normalize_mystem(text)
    else:
        tokens = text.split()
        # чтобы не доставать одно слово несколько раз,
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(tokens)
    total = len(tokens)
    vectors = np.zeros((len(words), dim))

    for i, word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v * (words[word] / total)  # просто умножаем вектор на частоту
        except Exception as e:
            #print(e)
            continue
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))

    return vector

Функция получения векторов заданного размера по заданной модели для нашего корпуса

In [27]:
def get_vectors_by_model(data, model, dim, is_mystem):

    X_text_1 = np.zeros((len(data['text_1_norm']), dim))
    X_text_2 = np.zeros((len(data['text_2_norm']), dim))
    for i, text in enumerate(data['text_1_norm'].values):
        X_text_1[i] = get_embedding(text, model, dim, is_mystem)
        if i % 100 == 0:
            print('processed text_1:', i)

    for i, text in enumerate(data['text_2_norm'].values):
        X_text_2[i] = get_embedding(text, model, dim, is_mystem)
        if i % 100 == 0:
            print('processed text2:', i)

    return X_text_1, X_text_2

Получим вектора при помощи rusvectores

In [142]:
#Раскомментировать, чтобы выполнить
#X_text_1_rusvectores, X_text_2_rusvectores = get_vectors_by_model(data, model_rusvectores, 300, True)
#Сохраним полученные вектора в pickle
#with open('X_text_1_rusvectores.pkl', 'wb') as f:
    #pickle.dump(X_text_1_rusvectores, f)

#Сохраним полученные вектора в pickle
#with open('X_text_2_rusvectores.pkl', 'wb') as f:
    #pickle.dump(X_text_2_rusvectores, f)

#Прочитаем из pickle
with open('X_text_1_rusvectores.pkl', 'rb') as f:
    X_text_1_rusvectores = pickle.load(f)

#Прочитаем из pickle
with open('X_text_2_rusvectores.pkl', 'rb') as f:
    X_text_2_rusvectores = pickle.load(f)

Получим вектора при помощи нашей модели

In [39]:
#Раскомментировать, чтобы выполнить
#X_text_1_fish, X_text_2_fish = get_vectors_by_model(data, model_fish, 50, False)
#Сохраним полученные вектора в pickle
#with open('X_text_1_fish.pkl', 'wb') as f:
    #pickle.dump(X_text_1_fish, f)

#Сохраним полученные вектора в pickle
#with open('X_text_2_fish.pkl', 'wb') as f:
    #pickle.dump(X_text_2_fish, f)

#Прочитаем из pickle
with open('X_text_1_fish.pkl', 'rb') as f:
    X_text_1_fish = pickle.load(f)

#Прочитаем из pickle
with open('X_text_2_fish.pkl', 'rb') as f:
    X_text_2_fish = pickle.load(f)

## Построим классификаторы на обеих моделях

Конкатенируем вектора

In [143]:
X_text_rusvectores = np.concatenate([X_text_1_rusvectores, X_text_2_rusvectores], axis=1)
X_text_fish = np.concatenate([X_text_1_fish, X_text_2_fish], axis=1)

In [174]:
y

array(['0', '0', '0', ..., '-1', '-1', '-1'], dtype=object)

При помощи кросс-валидации оценим качество классификации

In [178]:
clf_rusvectores = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_leaf=15,
                             class_weight='balanced')
scores_rusvectores = cross_val_score(clf_rusvectores, X_text_rusvectores, y, cv=5, scoring='f1_micro')

In [180]:
clf_fish = RandomForestClassifier(n_estimators=200, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
scores_fish = cross_val_score(clf_fish, X_text_fish, y, cv=5, scoring='f1_micro')

Значения, полученные на rusvectores:

In [179]:
scores_rusvectores, np.average(scores_rusvectores)

(array([0.47892191, 0.51278507, 0.52249135, 0.38642659, 0.40304709]),
 0.46073440273000604)

Значения, полученные на нашей модели

In [181]:
scores_fish, np.average(scores_fish)

(array([0.44989634, 0.48306842, 0.49965398, 0.35872576, 0.36357341]),
 0.4309835805757295)

## Выводы

Модель, обученная на небольшом корпусе специфической тематики (рыбная ловля), хуже справляется с перефразированием, чем модель, обученная на достаточно большом корпусе, собранном из интернета. При построении векторов было видно, что многие слова (в основном, географические названия) отсутствуют в корпусе fishkamchatka.

# Часть 2. Обучение по нескольким методам

## Преобразуем данные в вектора методом SVD

Построим матрицу "слова на документы"

In [89]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X_text_1_matrix = cv.fit_transform(data['text_1_norm'])
X_text_2_matrix = cv.fit_transform(data['text_2_norm'])

Разложим матрицу

In [91]:
X_text_1_svd = TruncatedSVD(50)
X_text_1_svd.fit(X_text_1_matrix)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [92]:
X_text_2_svd = TruncatedSVD(50)
X_text_2_svd.fit(X_text_2_matrix)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [76]:
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}

In [77]:
id2vec_svd_text1 = X_text_1_svd.components_.T
id2vec_svd_text2 = X_text_2_svd.components_.T

In [78]:
def most_similar(word, id2vec):
    similar = [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]
    return similar

In [85]:
most_similar('год', id2vec_svd_text1)

['год',
 'уголовный',
 'рана',
 'последний',
 'pussy',
 'иран',
 'сеть',
 'беспорядок',
 'папа',
 'глава']

In [86]:
most_similar('год', id2vec_svd_text2)

['год',
 '2014',
 'приговорить',
 'тюрьма',
 'посадить',
 'карта',
 'колония',
 'заключение',
 '2016',
 'впервые']

## Преобразуем данные в вектора методом NMF

In [ ]:
X_text_1_nmf = NMF(50)
X_text_1_nmf.fit(X_text_1_matrix)

In [103]:
X_text_2_nmf = NMF(50)
X_text_2_nmf.fit(X_text_2_matrix)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=50, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [105]:
id2vec_nmf_text1 = X_text_1_nmf.components_.T
id2vec_nmf_text2 = X_text_2_nmf.components_.T

In [106]:
most_similar("год", id2vec_nmf_text1)

['год',
 'уголовный',
 'официальный',
 'поддержка',
 'беспорядок',
 'рана',
 'папа',
 'иран',
 'последний',
 'авария']

In [107]:
most_similar("год", id2vec_nmf_text2)

['год',
 'приговорить',
 'карта',
 'тюрьма',
 'колония',
 '2014',
 'посадить',
 'вывести',
 'заключение',
 'виновник']

## Получим fasttext-модель из нашего корпуса

In [136]:
#раскомментировать, чтобы пересчитать
#model_fish_fasttext = gensim.models.FastText([text.split() for text in data_norm], size=50, min_n=4, max_n=8) 

In [182]:
#раскомментировать, чтобы пересчитать
#model_fish_fasttext.save('model_fish_fasttext.bin')

In [183]:
model_fish_fasttext = gensim.models.FastText.load('model_fish_fasttext.bin')

In [185]:
model_fish_fasttext.wv.most_similar('лосось')

[('лось', 0.9739271402359009),
 ('лососина', 0.9495588541030884),
 ('авось', 0.9270151257514954),
 ('лосев', 0.873853325843811),
 ('сосьва', 0.7208265662193298),
 ('лос-лагос', 0.701827347278595),
 ('лососёвый', 0.6500760316848755),
 ('горбуша', 0.6274489164352417),
 ('гм-сёмга', 0.6055312156677246),
 ('философия', 0.6042181253433228)]

In [186]:
model_fish_fasttext.wv.most_similar('лодка')

[('мотолодка', 0.9670103192329407),
 ('моторка', 0.9258214831352234),
 ('подлодка', 0.8962766528129578),
 ('сводка', 0.8745130300521851),
 ('катер', 0.8703102469444275),
 ('сковородка', 0.8615715503692627),
 ('сплав', 0.8609603047370911),
 ('наводка', 0.8487827181816101),
 ('руль-мотор', 0.8264052867889404),
 ('мотор', 0.8115783333778381)]

## Преобразуем данные в вектора методом fasttext

Используем модель, обученную на нашем корпусе

In [194]:
model_fish_fasttext = gensim.models.Word2Vec.load('model_fish_fasttext.bin')
X_text_1_fish_fasttext, X_text_2_fish_fasttext = get_vectors_by_model(data, model_fish_fasttext, 50, False)
#Сохраним полученные вектора в pickle
with open('X_text_1_fish_fasttext.pkl', 'wb') as f:
    pickle.dump(X_text_1_fish_fasttext, f)

#Сохраним полученные вектора в pickle
with open('X_text_2_fish_fasttext.pkl', 'wb') as f:
    pickle.dump(X_text_2_fish_fasttext, f)

#Прочитаем из pickle
with open('X_text_1_fish_fasttext.pkl', 'rb') as f:
    X_text_1_fish_fasttext = pickle.load(f)

#Прочитаем из pickle
with open('X_text_2_fish_fasttext.pkl', 'rb') as f:
    X_text_2_fish_fasttext = pickle.load(f)

c:\python35_64\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


processed text_1: 0
processed text_1: 100
processed text_1: 200
processed text_1: 300
processed text_1: 400
processed text_1: 500
processed text_1: 600
processed text_1: 700
processed text_1: 800
processed text_1: 900
processed text_1: 1000
processed text_1: 1100
processed text_1: 1200
processed text_1: 1300
processed text_1: 1400
processed text_1: 1500
processed text_1: 1600
processed text_1: 1700
processed text_1: 1800
processed text_1: 1900
processed text_1: 2000
processed text_1: 2100
processed text_1: 2200
processed text_1: 2300
processed text_1: 2400
processed text_1: 2500
processed text_1: 2600
processed text_1: 2700
processed text_1: 2800
processed text_1: 2900
processed text_1: 3000
processed text_1: 3100
processed text_1: 3200
processed text_1: 3300
processed text_1: 3400
processed text_1: 3500
processed text_1: 3600
processed text_1: 3700
processed text_1: 3800
processed text_1: 3900
processed text_1: 4000
processed text_1: 4100
processed text_1: 4200
processed text_1: 4300


Конкатенируем вектора

In [195]:
X_text_fish_fasttext = np.concatenate([X_text_1_fish_fasttext, X_text_2_fish_fasttext], axis=1)

## Обучим классификатор

In [117]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.4, max_features=1000, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [120]:
X_text_1_svd_text = X_text_1_svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd_text = X_text_2_svd.fit_transform(tfidf.transform(data['text_2_norm']))

X_svd_text = np.concatenate([X_text_1_svd_text, X_text_2_svd_text], axis=1)
X_svd_text.shape

(7227, 100)

In [121]:
X_text_1_nmf_text = X_text_1_nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf_text = X_text_2_nmf.fit_transform(tfidf.transform(data['text_2_norm']))

X_nmf_text = np.concatenate([X_text_1_nmf_text, X_text_2_nmf_text], axis=1)
X_nmf_text.shape

(7227, 100)

Скомбинируем все полученные признаки.

In [208]:
X_text_combined = np.concatenate([X_svd_text, X_nmf_text, X_text_fish, X_text_rusvectores, X_text_fish_fasttext], axis=1)
X_text_combined.shape

(7227, 1000)

In [209]:
clf_combined = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_leaf=15,
                             class_weight='balanced')
scores_combined = cross_val_score(clf_combined, X_text_combined, y, cv=5, scoring='f1_micro')

Результат

In [210]:
scores_combined, np.average(scores_combined)

(array([0.46302695, 0.50656531, 0.52041522, 0.40166205, 0.4065097 ]),
 0.4596358459827622)

## Выводы

Результат, полученный на скомбинированных векторах, мало отличается от результата на rusvectores, что говорит, вероятно, о высоком качестве модели, обученной на большом корпусе